
# 4.1.4 — Monte Carlo (bases) : martingale & mouvement brownien

## 🎯 Objectifs
- Comprendre la **méthode de Monte Carlo** sur des fonctionnelles simples du **mouvement brownien** $W_t$.
- Vérifier numériquement des **identités en loi** (martingale, normalité, variance).
- Mettre en œuvre des **techniques simples** : antithétiques, estimation de l'erreur, convergence en $\mathcal{O}(1/\sqrt{M})$.
- Savoir estimer des quantités comme $\mathbb{E}[f(W_T)]$ et des **probabilités de franchissement** élémentaires.

---

## 1) Rappels : Brownien & martingale

- $W_0=0$, incréments indépendants, $W_T\sim \mathcal{N}(0,T)$.
- Martingales classiques :
  - $W_t$ est martingale.
  - $W_t^2 - t$ est martingale.
  - $\exp\!\big(\lambda W_t - \tfrac12 \lambda^2 t\big)$ est martingale (exponentielle de Doléans).

Nous allons **vérifier numériquement** quelques égalités d'espérance avec Monte Carlo.



## 2) Monte Carlo : $\mathbb{E}[f(W_T)]$ et estimation d'erreur

Pour $W_T\sim \mathcal{N}(0,T)$, on peut simuler directement $W_T=\sqrt{T}\,Z$ avec $Z\sim\mathcal{N}(0,1)$ et approximer
$$
\mathbb{E}[f(W_T)] \approx \frac{1}{M}\sum_{m=1}^M f(W_T^{(m)}).
$$
Erreur statistique $\approx \mathrm{SE} = \hat{\sigma}/\sqrt{M}$.  
On illustre la **convergence** quand $M$ augmente.


In [ ]:

import numpy as np

def mc_E_f_WT(f, T=1.0, M=100_000, seed=0):
    rng = np.random.default_rng(seed)
    WT = np.sqrt(T) * rng.standard_normal(M)
    vals = f(WT)
    mean = vals.mean()
    se = vals.std(ddof=1)/np.sqrt(M)
    return mean, se

# Exemples : f(x)=x^2, f(x)=exp(x)
for f in [
    lambda x: x**2,
    lambda x: np.exp(x),
]:
    m, se = mc_E_f_WT(f, T=1.0, M=200_000, seed=42)
    print("MC mean≈", m, "  SE≈", se)



> **Remarque** : pour $f(x)=x^2$ et $T=1$, on sait que $\mathbb{E}[W_1^2]=1$.  
Pour $f(x)=e^x$, $\mathbb{E}[e^{W_1}] = e^{1/2}$ (moment de la loi normale).  
Vérifiez la proximité avec l'estimation Monte Carlo.



## 3) Variance reduction : **antithétiques**

Idée : si on simule $Z$ alors on simule aussi $-Z$ ; on moyenne $f(\sqrt{T}Z)$ et $f(-\sqrt{T}Z)$.  
Souvent, cela **réduit la variance** (symétrie).


In [ ]:

import numpy as np

def mc_antithetic(f, T=1.0, M=100_000, seed=0):
    rng = np.random.default_rng(seed)
    Z = rng.standard_normal(M//2)
    WT_plus = np.sqrt(T)*Z
    WT_minus = -WT_plus
    vals = 0.5*(f(WT_plus) + f(WT_minus))
    mean = vals.mean()
    se = vals.std(ddof=1)/np.sqrt(M//2)  # M/2 moyennes indépendantes
    return mean, se

m1, se1 = mc_E_f_WT(lambda x: np.exp(x), T=1.0, M=200_000, seed=1)
m2, se2 = mc_antithetic(lambda x: np.exp(x), T=1.0, M=200_000, seed=1)

(m1, se1), (m2, se2)  # comparez les SE



## 4) Martingales simples : tests numériques

- $\mathbb{E}[W_T]=0$
- $\mathbb{E}[W_T^2 - T]=0$
- $\mathbb{E}[\exp(\lambda W_T - \tfrac12\lambda^2 T)]=1$

On vérifie au Monte Carlo.


In [ ]:

import numpy as np

def test_martingales(T=1.0, M=200_000, lam=0.7, seed=2):
    rng = np.random.default_rng(seed)
    WT = np.sqrt(T) * rng.standard_normal(M)
    m1 = WT.mean()
    m2 = (WT**2 - T).mean()
    m3 = np.exp(lam*WT - 0.5*(lam**2)*T).mean()
    return m1, m2, m3

test_martingales()



## 5) Probabilité de franchissement (basique)

Pour un **niveau** $a>0$, la probabilité $\mathbb{P}(\max_{0\le s\le T} W_s \ge a)$ admet une formule (méthode de réflexion) :
$$
\mathbb{P}(\max_{0\le s\le T} W_s \ge a) = 2\,\mathbb{P}(W_T \ge a) = 2 \big(1-\Phi(a/\sqrt{T})\big).
$$
On vérifie numériquement via des trajectoires discrètes.


In [ ]:

import numpy as np

def brownian_paths(n_paths=50_000, T=1.0, N=500, seed=3):
    rng = np.random.default_rng(seed)
    dt = T/N
    dW = rng.normal(0.0, np.sqrt(dt), size=(n_paths, N))
    W = np.cumsum(dW, axis=1)
    W = np.hstack((np.zeros((n_paths,1)), W))
    return W  # shape: (n_paths, N+1)

def crossing_prob(a=1.0, T=1.0, n_paths=50_000, N=500, seed=3):
    W = brownian_paths(n_paths, T, N, seed)
    hit = (W.max(axis=1) >= a).mean()
    return hit

# Comparaison MC vs formule
a, T = 1.0, 1.0
mc = crossing_prob(a, T, n_paths=30_000, N=600, seed=4)
from math import erf, sqrt
import numpy as np
def Phi(x):  # CDF N(0,1)
    return 0.5*(1+erf(x/np.sqrt(2)))
theo = 2*(1-Phi(a/np.sqrt(T)))
mc, theo



---

## 📌 À retenir
- $W_T\sim \mathcal{N}(0,T)$ : on peut simuler **directement** sans trajectoire si on ne dépend que de $W_T$.
- L’erreur MC décroît comme $\mathcal{O}(1/\sqrt{M})$ ; **antithétiques** aident souvent.
- Tester des **martingales** est un bon sanity check.
- Franchissement : formule simple via **réflexion** pour le Brownien standard.

## ✍️ Exercices (simples mais importants)
1. Vérifier $\mathbb{E}[\exp(\lambda W_T - \tfrac12\lambda^2T)]=1$ pour plusieurs valeurs de $\lambda$ et $T$ ; tracer l’erreur vs $M$.  
2. Implémenter une **borne de confiance** à 95% sur l’estimateur MC.  
3. Estimer $\mathbb{E}[(W_T^+)^p]$ pour $p=1,2$ et comparer au calcul analytique (moment demi-normal).
